In [10]:
import pickle

#author: kfransen
#modeling of terpolymer library data with optimized models and model structures from the original library


#optimized library selections:
#list of [num estimators, max depth, max leaf nodes]


#original_rf selected for cross_val>0.7 and test score >0.72
original_rf=[[8,4,8],[16,16,128],[16,128,512],[32,256,8]]
#reduced_rf selected for cross_val>0.81
reduced_rf=[[8,2,8],[16,4,128],[64,256,8],[256,4,256]]
#original_SDG selected for
original_SDG=[['hinge',0.1],['log_loss',0.01],['perceptron',0.001]]
#reduced SDG selected for
reduced_SDG=[['hinge',0.1],['log_loss',0.1],['perceptron',0.0001]]


In [2]:
#import block

#imports
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
import os.path
import json
import seaborn as sns
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.rdFingerprintGenerator import GetRDKitFPGenerator
import ot
import csv
from pickle import dump
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import pairwise_distances
from scipy.optimize import minimize
from scipy.stats import bernoulli
from scipy.special import expit as sigmoid
from sklearn.datasets import make_moons
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, RationalQuadratic
from multiprocessing import Pool

#import EMD calculation from Jiale's project
from Polymer_Similarity import EMD_Calculation_pot as EMD

In [245]:
# function definition block
def str_to_list(string):
    #string='"'+str(string)+'"'
    counter=string.count(',')
    #print(counter)
    list=[]
    for i in range(counter):
        if counter ==1:
            string=string[1:len(string)-1]
        if i==0 and counter>1:
            j=string.index(',')
            list+=[string[2:j-1]]
            string=string[j+1:len(string)-1]
            if string[0]==' ':
                string=string[1:]
        elif i==counter-1:
            j=string.index(',')
            list+=[string[1:j-1]]
            if string[j+1]==' ':
                list+=[string[j+3:len(string)-1]]
            else:
                list+=[string[j+2:len(string)-1]]
        else:
            j=string.index(',')
            list+=[string[1:j-1]]
            if string[0]==' ':
                string=string[j+2:]
            else:
                string=string[j+1:]
        #print(string)

    return list

def concatenate_indices(indices,leave_out):
    #indices is a list of the split arrays, leave out is the index number of the test set
    newlist=[]
    test=None
    for i in range(len(indices)):
        if i==leave_out:
            test=indices[i]
        else:
            newlist+=[indices[i]]
    train=newlist[0]
    for i in range(1,len(newlist)):
        train=np.concatenate((train,newlist[i]),0)
    return [train,test]

def rf_cross_val(C, xs, ys,testx,testy,name):
    #C should be [n_estimators, max_depth,max_leaf_nnodes]
    #xs and ys give a list of the different dataset splices - must be same length, indices in each must match
        #this should be the full training set (will iterate which set is being used as the validation set for cross validation
    scores = []
    for i in range(0, len(xs)):
        X = concatenate_indices(xs, i)
        Y = concatenate_indices(ys, i)
        model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1,max_leaf_nodes=C[2])
        model.fit(X[0], Y[0])
        scores += [model.score(X[1], Y[1])]
    score_array = np.array(scores)
    mean = np.mean(score_array)
    std = np.std(score_array)

    #now evaluate against unseen testing data
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1,
                                   max_leaf_nodes=C[2])
    model.fit(X[0], Y[0])
    eval=model.score(testx,testy)
    with open("./240527_with_Terpolymers_Optimization Models/RF_"+str(name)+"_[n_estim,max_depth,max_leaf]_"+str(C)+".pkl", "wb") as f:
        dump(model, f, protocol=5)
    return [mean, std, eval]

def train_rf(C,xs,ys):
    #C should be [n_estimators, max_depth,max_leaf_nnodes]
    #xs and ys should be list of the training data (no validation set, test set should be left out)
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = RandomForestClassifier(n_estimators=C[0], max_depth=C[1], min_samples_split=2, min_samples_leaf=1, max_leaf_nodes=C[2],random_state=42)
    model.fit(X[0], Y[0])
    return model

def update_rf(old_model,C,xs,ys):
    #this takes an existing trained rf and fits additional trees given additional training data
    #C should be the number of additional estimators (trees) to fit
    #xs and ys should be list of the training data (no validation set, test set should be left out)
    n_estims=len(old_model.estimators_)

    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model_list=[]
    estimator_amounts=[]
    model = old_model.set_params(n_estimators=C+n_estims,warm_start=True)
    model.fit(X[0], Y[0])
    return model


def lin_cross_val(C, xs, ys,xtest,ytest,name):
    #C should be [loss, alpha]
    #xs and ys give a list of the different dataset splices - must be same length, indices in each must match
        #this should be the full training set (will iterate which set is being used as the validation set for cross validation
    scores = []
    for i in range(0, len(xs)):
        X = concatenate_indices(xs, i)
        Y = concatenate_indices(ys, i)
        model =SGDClassifier(loss=C[0], alpha=C[1],shuffle=True)
        model.fit(X[0], Y[0])
        scores += [model.score(X[1], Y[1])]
    score_array = np.array(scores)
    mean = np.mean(score_array)
    std = np.std(score_array)
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = SGDClassifier(loss=C[0], alpha=C[1], shuffle=True)
    model.fit(X[0],Y[0])
    eval=model.score(xtest,ytest)
    #save the model
    with open("./240527_with_Terpolymers_Optimization Models/SGD_"+str(name)+"_[loss,alpha]_"+str(C)+".pkl", "wb") as f:
        dump(model, f, protocol=5)
    return [mean, std,eval]

def train_lin(C, xs,ys):
    X = concatenate_indices(xs, None)
    Y = concatenate_indices(ys, None)
    model = SGDClassifier(loss=C[0], alpha=C[1], shuffle=True,random_state=42)
    model.fit(X[0], Y[0])
    return model

def update_lin(old_model, xs, ys):
    #xs are additional new data that needs to be fit
    #ys are labels corresponding to xs
    class_biodeg=[0,1] #pretty sure since we use 0 or 1 this is all we've got
    model=old_model.partial_fit(xs,ys,classes=class_biodeg)
    return model

def get_EMD_array(sets,basis,EMD_values=["RDKFingerprint","Tanimoto",5,512]):
    #sets = list of sets (pd dataframes) that need EMD calculations against the basis
    #if basis=None then the concatenation of the sets is used as basis. If not, give polymer basis
    #EMD_values gives [embedding_function, similarity_score_function, radius, num_bits]

    EMD_arrays=[]
    for k in range(len(sets)):
        col = []
        for i in range(len(sets[k].index)):
            row = []
            for j in range(len(basis.index)):
                X = sets[k]['SMILES_List'][i]
                #print(X)
                if isinstance(X,str):
                    X = str_to_list(X)
                #print(X)
                Y = sets[k]['Ratio_List'][i]
                #print(basis['SMILES_List'])
                #print("basis",basis['SMILES_List'][j])
                Z = basis['SMILES_List'][j]
                #print(Z)
                if isinstance(Z,str):
                    Z = str_to_list(Z)
                #print(Z)
                W = basis['Ratio_List'][j]
                # W=str_to_list(W)
                #print(X, Y, Z, W)
                score = EMD(
                    query_smiles_list=X,
                    query_smiles_weight_list=Y,
                    target_smiles_list=Z,
                    target_smiles_weight_list=W,
                    embedding_function=EMD_values[0],
                    similarity_score_function=EMD_values[1],
                    radius=EMD_values[2],
                    num_bits=EMD_values[3],
                )
                row += [score]
            col += [row]
        # save the list of lists as a npy array
        formed = np.array(col)
        EMD_arrays+=[formed]
    return EMD_arrays


In [168]:
#first generate all the different sets of splits for the terpolymer library
#test set of 1/6th of the library that is otherwise untouched (pull first)
#Then two pathways forward
#   1. split remainder into 5 other sets that can be used for training
#   2. split each of the 5 sets into groups of 2 (part A and part B) (so that there are splits of 10% of the 5/6ths of the library that can be used for training

#balance fraction of degrading and non-degrading samples for the subsets and DMT/DMI vs not DMT/DMI
ter_Julia=pd.read_csv('./Library_Data_Julia_Ratios.csv',header=0)
ter_other=pd.read_csv('./Library_Data_Final_Ratios.csv',header=0)
ter_Julia_deg=ter_Julia.loc[ter_Julia['Biodegradable']=='Y']
ter_Julia_non=ter_Julia.loc[ter_Julia['Biodegradable']=='N']
ter_other_deg=ter_other.loc[ter_other['Biodegradable']=='Y']
ter_other_non=ter_Julia.loc[ter_other['Biodegradable']=='N']
for i in range(6):
    fraction=1/(6-i)
    degrading_julia=ter_Julia_deg.sample(frac=fraction)
    degrading_other=ter_other_deg.sample(frac=fraction)
    nondeg_julia=ter_Julia_non.sample(frac=fraction)
    nondeg_other=ter_other_non.sample(frac=fraction)
    ter_julia_deg=ter_Julia_deg.drop(degrading_julia.index)
    ter_other_deg=ter_other_deg.drop(degrading_other.index)
    ter_Julia_non=ter_Julia_non.drop(nondeg_julia.index)
    ter_other_non=ter_other_non.drop(nondeg_other.index)
    degrading_julia_A=degrading_julia.sample(frac=0.5)
    degrading_julia_B=degrading_julia.drop(degrading_julia_A.index)
    nondeg_julia_A=nondeg_julia.sample(frac=0.5)
    nondeg_julia_B=nondeg_julia.drop(nondeg_julia_A.index)
    degrading_other_A=degrading_other.sample(frac=0.5)
    degrading_other_B=degrading_other.drop(degrading_other_A.index)
    nondeg_other_A=nondeg_other.sample(frac=0.5)
    nondeg_other_B=nondeg_other.drop(nondeg_other_A.index)
    all=pd.concat([degrading_julia,degrading_other,nondeg_julia,nondeg_other])
    set_A=pd.concat([degrading_julia_A,degrading_other_A,nondeg_julia_A,nondeg_other_A])
    set_B=pd.concat([degrading_julia_B,degrading_other_B,nondeg_julia_B,nondeg_other_B])
    all.to_csv(path_or_buf='./Terpolymer_Splits/split_'+str(i)+'.csv')
    set_A.to_csv(path_or_buf='./Terpolymer_Splits/split_'+str(i)+'_A'+'.csv')
    set_B.to_csv(path_or_buf='./Terpolymer_Splits/split_'+str(i)+'_B'+'.csv')


In [169]:
#write npy file of the ys for each of the ys for each set

for i in range(6):
    all=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'.csv',header=0)
    set_A=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_A'+'.csv',header=0)
    set_B=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_B'+'.csv',header=0)
    all_biodeg=all['Biodegradable'].replace('N',0)
    all_biodeg=all_biodeg.replace('Y',1)
    set_A_biodeg=set_A['Biodegradable'].replace('N',0)
    set_A_biodeg=set_A_biodeg.replace('Y',1)
    set_B_biodeg=set_B['Biodegradable'].replace('Y',1)
    set_B_biodeg=set_B_biodeg.replace('N',0)
    all_biodeg_np=all_biodeg.to_numpy()
    set_A_biodeg_np=set_A_biodeg.to_numpy()
    set_B_biodeg_np=set_B_biodeg.to_numpy()
    print(set_A_biodeg_np)
    np.save('./Terpolymer_Splits/split'+str(i)+'_ys',all_biodeg_np,allow_pickle=False)
    np.save('./Terpolymer_Splits/split'+str(i)+'_A_ys',set_A_biodeg_np,allow_pickle=False)
    np.save('./Terpolymer_Splits/split'+str(i)+'_B_ys',set_B_biodeg_np,allow_pickle=False)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0]


/var/folders/t5/wsvlnfc52kd7dmby8bpwq66r0000gn/T/ipykernel_47108/3563030578.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  all_biodeg=all_biodeg.replace('Y',1)
/var/folders/t5/wsvlnfc52kd7dmby8bpwq66r0000gn/T/ipykernel_47108/3563030578.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  set_A_biodeg=set_A_biodeg.replace('Y',1)
/var/folders/t5/wsvlnfc52kd7dmby8bpwq66r0000gn/T/ipykernel_47108/3563030578.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the 

In [198]:
def get_smiles_and_ratios_list(dataframe):
    names=dataframe['Polymer_Name_P'].to_list()
    ratio1=dataframe['Ratio 1'].to_list()
    ratio2=dataframe['Ratio 2'].to_list()
    ratio3=dataframe['Ratio 3'].to_list()
    smiles=[]
    ratios=[]
    monomers=pd.read_csv('./Monomer Info_DMTDMI.csv',header=0)
    for i in range(len(names)):
        name=names[i]
        smile=[]
        smile+=[str(monomers.loc[monomers['Polymer Name Abbr.']==name[0:3],'SMILES_Monomers'].item())]
        smile+=[str(monomers.loc[monomers['Polymer Name Abbr.']==name[6:9],'SMILES_Monomers'].item())]
        smile+=[str(monomers.loc[monomers['Polymer Name Abbr.']==name[12:15],'SMILES_Monomers'].item())]
        #print(smile)
        smiles+=[smile]
        #print(smiles)
        ratios+=[[ratio1[i]/2,ratio2[i]/2,ratio3[i]/2]]
    return[smiles,ratios]

In [199]:
#load the models that were trained on the original library and reduced library
#original_rf selected for cross_val>0.7 and test score >0.72
original_rf=[[8,4,8],[16,16,128],[16,128,512],[32,256,8]]
#reduced_rf selected for cross_val>0.81
reduced_rf=[[8,2,8],[16,4,128],[64,256,8],[256,4,256]]
#original_SDG selected for
original_SDG=[['hinge',0.1],['log_loss',0.01],['perceptron',0.001]]
#reduced SDG selected for
reduced_SDG=[['hinge',0.1],['log_loss',0.1],['perceptron',0.0001]]
with open('optimal_model_evaluations.csv', 'w', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Dataset and Model Type','Model Parameters','Score against Terpolymer test set'])
#make EMD matrix of ter polymer test set (set 5) against original training set and reduced training set
ter_test=pd.read_csv('./Terpolymer_Splits/split_5.csv',header=0)
test_ter_list=get_smiles_and_ratios_list(ter_test)



In [193]:
def get_EMD_array_lists(sets,ratios,basis,EMD_values=["RDKFingerprint","Tanimoto",5,512]):
    #sets = list of sets (each set is a list of smiles lists dataframes) that need EMD calculations against the basis
    #ratios is a list of ratio lists corresponding to the sets
    #if basis=None then the concatenation of the sets is used as basis. If not, give polymer basis
    #EMD_values gives [embedding_function, similarity_score_function, radius, num_bits]

    EMD_arrays=[]
    for k in range(len(sets)):
        col = []
        for i in range(len(sets[k])):
            row = []
            for j in range(len(basis.index)):
                X = sets[k][i]
                print(X)
                Y = ratios[k][i]
                #print(basis['SMILES_List'])
                #print("basis",basis['SMILES_List'][j])
                Z = basis['SMILES_List'][j]
                #print(Z)
                Z = str_to_list(Z)
                W = basis['Ratio_List'][j]
                # W=str_to_list(W)
                #print(X, Y, Z, W)
                score = EMD(
                    query_smiles_list=X,
                    query_smiles_weight_list=Y,
                    target_smiles_list=Z,
                    target_smiles_weight_list=W,
                    embedding_function=EMD_values[0],
                    similarity_score_function=EMD_values[1],
                    radius=EMD_values[2],
                    num_bits=EMD_values[3],
                )
                row += [score]
            col += [row]
        # save the list of lists as a npy array
        formed = np.array(col)
        EMD_arrays+=[formed]
    return EMD_arrays

In [178]:
#load original and reduced data
os0=pd.read_csv('./ML_splits/original_split0.csv',header=0)
os1=pd.read_csv('./ML_splits/original_split1.csv',header=0)
os2=pd.read_csv('./ML_splits/original_split2.csv',header=0)
os3=pd.read_csv('./ML_splits/original_split3.csv',header=0)
os4=pd.read_csv('./ML_splits/original_split4.csv',header=0)
os5=pd.read_csv('./ML_splits/original_split5.csv',header=0)
y_set0=np.load('./ML_splits/Biodeg_fororiginal_split0_vsfull_library.npy')
y_set1=np.load('./ML_splits/Biodeg_fororiginal_split1_vsfull_library.npy')
y_set2=np.load('./ML_splits/Biodeg_fororiginal_split2_vsfull_library.npy')
y_set3=np.load('./ML_splits/Biodeg_fororiginal_split3_vsfull_library.npy')
y_set4=np.load('./ML_splits/Biodeg_fororiginal_split4_vsfull_library.npy')
y_set5=np.load('./ML_splits/Biodeg_fororiginal_split5_vsfull_library.npy')
os_train=[os0,os1,os2,os3,os4] #the training data for the cross validation/full model
os_train_pd=pd.concat(os_train).reset_index()
rl0=pd.read_csv('./Reduced_ML_Splits/reduced_split0.csv',header=0)
rl1=pd.read_csv('./Reduced_ML_Splits/reduced_split1.csv',header=0)
rl2=pd.read_csv('./Reduced_ML_Splits/reduced_split2.csv',header=0)
rl3=pd.read_csv('./Reduced_ML_Splits/reduced_split3.csv',header=0)
rl4=pd.read_csv('./Reduced_ML_Splits/reduced_split4.csv',header=0)
rl5=pd.read_csv('./Reduced_ML_Splits/reduced_split5.csv',header=0)
rl_train=[rl0,rl1,rl2,rl3,rl4] #the training data used for cross validation/full model
rl_train_pd=pd.concat(rl_train).reset_index()


In [179]:
#get the EMD arrays for terpolymer testing vs original and reduced libraries
tertest_emd_vs_orig=get_EMD_array_lists([test_ter_list[0]],[test_ter_list[1]],os_train_pd)
tertest_emd_vs_reduc=get_EMD_array_lists([test_ter_list[0]],[test_ter_list[1]],rl_train_pd)
np.save('./Terpolymer_Splits/EMD_split'+str(5)+'_vs_orig',tertest_emd_vs_orig,allow_pickle=False)
np.save('./Terpolymer_Splits/EMD_split'+str(5)+'_vs_reduc',tertest_emd_vs_reduc,allow_pickle=False)

['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(ccc1)cc1C(=O)', 'OCCCCCCO']
['C(=O)COCCOCC(=O)', 'C(=O)c(cc

In [58]:
#load all the models of interest to test against and record performance vs ter_polymer test set
import pickle
Xval=np.load('./Terpolymer_Splits/EMD_split5_vs_orig.npy')[0]
print(Xval)
Yval=np.load('./Terpolymer_Splits/split5_ys.npy')
for i in range(len(original_rf)):
    C=original_rf[i]
    with open("./240523_Optimization_Models/RF_os_estim_"+str(C[0])+"_depth_"+str(C[1])+"_leafnodes_"+str(C[2])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['RF_Original',str(C),value])
Xval_r=np.load('./Terpolymer_Splits/EMD_split5_vs_reduc.npy')[0]

for i in range(len(reduced_rf)):
    C=reduced_rf[i]
    with open("./240523_Optimization_Models/RF_rl_estim_"+str(C[0])+"_depth_"+str(C[1])+"_leafnodes_"+str(C[2])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval_r,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['RF_Reduced',str(C),value])


[[0.57930624 0.62080046 0.87122345 ... 0.78859193 0.79883838 0.73437108]
 [0.37367443 0.63194855 0.82610905 ... 0.65741268 0.80231808 0.90195713]
 [0.53553107 0.76516824 0.82444558 ... 0.74303936 0.82571962 0.90526887]
 ...
 [0.53592317 0.54885791 0.82925429 ... 0.84916024 0.80970172 0.86302156]
 [0.4696234  0.61988574 0.86328483 ... 0.80776274 0.77171768 0.88482908]
 [0.45793267 0.62704258 0.87832992 ... 0.78080723 0.81512066 0.88990407]]


In [59]:
#original_SDG selected for
original_SDG=[['hinge',0.1],['log_loss',0.01],['perceptron',0.001]]
#reduced SDG selected for
reduced_SDG=[['hinge',0.1],['log_loss',0.1],['perceptron',0.0001]]
for i in range(len(original_SDG)):
    C=original_SDG[i]
    with open("./240523_Optimization_Models/SGD_os"+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['SDG_Original',str(C),value])
Xval_r=np.load('./Terpolymer_Splits/EMD_split5_vs_reduc.npy')[0]

for i in range(len(reduced_SDG)):
    C=reduced_SDG[i]
    with open("./240523_Optimization_Models/SGD_rl"+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "rb") as f:
        model=pickle.load(f)
    value=model.score(Xval_r,Yval)
    with open('optimal_model_evaluations.csv', 'a', newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['SDG_Reduced',str(C),value])

In [200]:
#generate all the possible EMD 'landscapes' that we may be interested in from the ter polymer splits + the original library and reduced library
original_df=os_train_pd[['SMILES_List','Ratio_List']]
biodeg_original_np=os_train_pd['Biodegradability'].to_numpy()
reduced_df=rl_train_pd[['SMILES_List','Ratio_List']]
biodeg_reduced_np=rl_train_pd['Biodegradability'].to_numpy()
original_total=original_df
reduced_total=reduced_df
original_incr_ter=[original_df] #list of pd dataframes that have the increasing basis, starting with only original library
reduced_incr_ter=[reduced_df]
#list of np arrays that correspond to list of pd dataframes for training the models
original_np=biodeg_original_np
reduced_np=biodeg_reduced_np
original_incr_ter_ys=[]
reduced_incr_ter_ys=[]
#note- we only need to load and run the training datasets
for i in range(5):
    #print(i)
    A_pd=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_A.csv',header=0)
    B_pd=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_B.csv',header=0)
    A_np=np.load('./Terpolymer_Splits/split'+str(i)+'_A_ys.npy')
    B_np=np.load('./Terpolymer_Splits/split'+str(i)+'_B_ys.npy')
    values_A=get_smiles_and_ratios_list(A_pd)
    #print(values_A[0])
    values_B=get_smiles_and_ratios_list(B_pd)
    #print(values_B)
    A_newpd=pd.DataFrame({'SMILES_List':values_A[0],'Ratio_List':values_A[1]})
    #print(A_newpd)
    B_newpd=pd.DataFrame({'SMILES_List':values_B[0],'Ratio_List':values_B[1]})
    #print(original_total)
    original_total=pd.concat((original_total,A_newpd)).reset_index(drop=True)
    #print(original_total)
    reduced_total=pd.concat([reduced_total,A_newpd]).reset_index(drop=True)
    original_incr_ter+=[original_total]
    reduced_incr_ter+=[reduced_total]
    original_total=pd.concat([original_total,B_newpd]).reset_index(drop=True)
    original_incr_ter+=[original_total]
    reduced_total=pd.concat([reduced_total,B_newpd]).reset_index(drop=True)
    reduced_incr_ter+=[reduced_total]
    original_np=np.concatenate((original_np,A_np))
    original_incr_ter_ys+=[original_np]
    reduced_np=np.concatenate((reduced_np,A_np))
    reduced_incr_ter_ys+=[reduced_np]
    original_np=np.concatenate((original_np,B_np))
    original_incr_ter_ys+=[original_np]
    reduced_np=np.concatenate((reduced_np,B_np))
    reduced_incr_ter_ys+=[reduced_np]
with open('./Terpolymer_Splits/original_plus_incr_ter_pd_list.pkl',"wb") as f:
    dump(original_incr_ter,f,protocol=5)
with open('./Terpolymer_Splits/original_plus_incr_ter_ys_np_list.pkl',"wb") as f:
    dump(original_incr_ter_ys,f,protocol=5)
with open('./Terpolymer_Splits/reduced_plus_incr_ter_pd_list.pkl',"wb") as f:
    dump(reduced_incr_ter,f,protocol=5)
with open('./Terpolymer_Splits/reduced_plus_incr_ter_ys_np_list.pkl',"wb") as f:
    dump(reduced_incr_ter_ys,f,protocol=5)



In [217]:
#dataframe os5 is the original library test
#dataframe rl5 is the reduced library test

#make the 3 different test set evaluations
original_tests=os5[['SMILES_List','Ratio_List']]
original_tests_ys_np=os5['Biodegradability'].to_numpy()
reduced_tests=rl5[['SMILES_List','Ratio_List']]
reduced_tests_ys_np=rl5['Biodegradability'].to_numpy()

ter_test_pd=pd.DataFrame({'SMILES_List':test_ter_list[0],'Ratio_List':test_ter_list[1]})
Yval=np.load('./Terpolymer_Splits/split5_ys.npy')

original_test_pds=[original_tests,ter_test_pd,pd.concat([original_tests,ter_test_pd]).reset_index(drop=True)]
original_test_ylist=[original_tests_ys_np,Yval,np.concatenate([original_tests_ys_np,Yval])]
reduced_test_pds=[reduced_tests,ter_test_pd,pd.concat([reduced_tests,ter_test_pd]).reset_index(drop=True)]
reduced_test_ylist=[reduced_tests_ys_np,Yval,np.concatenate([reduced_tests_ys_np,Yval])]




In [218]:
#get the training and test X arrays (we got the y arrays above)

original_emd_train=[] #this should end up being a list of training arrays
original_emd_test=[] #this ends up being a list of list since for each basis there is a list of test sets
for b in original_incr_ter:
    print(b)
    array=get_EMD_array([b],b)
    original_emd_train+=[array[0]]
    array=get_EMD_array(original_test_pds,b)
    original_emd_test+=[array]
#dump all the data into pickles
with open('./Terpolymer_Splits/original_plus_incr_ter_train_emd_list.pkl',"wb") as f:
    dump(original_emd_train,f,protocol=5)
with open('./Terpolymer_Splits/original_plus_incr_ter_test_emd_list.pkl',"wb") as f:
    dump(original_emd_test,f,protocol=5)
with open('./Terpolymer_Splits/original_plus_incr_ter_testys_np_list.pkl',"wb") as f:
    dump(original_test_ylist,f,protocol=5)



reduced_emd_train=[] #this should end up being a list of training arrays
reduced_emd_test=[] #this ends up being a list of list since for each basis there is a list of test sets
for b in reduced_incr_ter:
    print(b)
    array=get_EMD_array([b],b)
    reduced_emd_train+=[array[0]]
    array=get_EMD_array(reduced_test_pds,b)
    reduced_emd_test+=[array]
#dump all the data into pickles
with open('./Terpolymer_Splits/reduced_plus_incr_ter_train_emd_list.pkl',"wb") as f:
    dump(reduced_emd_train,f,protocol=5)
with open('./Terpolymer_Splits/reduced_plus_incr_ter_test_emd_list.pkl',"wb") as f:
    dump(reduced_emd_test,f,protocol=5)
with open('./Terpolymer_Splits/reduced_plus_incr_ter_testys_np_list.pkl',"wb") as f:
    dump(reduced_test_ylist,f,protocol=5)

                                           SMILES_List Ratio_List
0               ['C(=O)c(cc1)ccc1C(=O)', 'OCCCCCCCCO']     [1, 1]
1    ['C(=O)c(cc1)ccc1C(=O)', 'Oc1c(C)cc(cc1(C))C(C...     [1, 1]
2    ['C(=O)C(=O)', 'Oc1ccc(cc1)C(C)(c2ccccc2)c3ccc...     [1, 1]
3                           ['C(=O)CC(=O)', 'OCC=CCO']     [1, 1]
4                       ['C(=O)C(=O)', 'OCCCCCCCCCCO']     [1, 1]
..                                                 ...        ...
541        ['C(=O)CCCCC(CCCC)O', 'C(=O)C=CCC(CCCCC)O']     [2, 1]
542        ['C(=O)CCCCC(CCCC)O', 'C(=O)CCCC(CCCCCC)O']     [2, 1]
543                  ['C(=O)COC(=O)CO', 'C(=O)CCCCCO']     [2, 1]
544                     ['C(=O)CCO', 'C(=O)COC(=O)CO']     [2, 1]
545         ['C(=O)C(C)OC(=O)C(C)O', 'C(=O)COC(=O)CO']     [2, 1]

[546 rows x 2 columns]
                                           SMILES_List         Ratio_List
0               ['C(=O)c(cc1)ccc1C(=O)', 'OCCCCCCCCO']             [1, 1]
1    ['C(=O)c(cc1)ccc1C(=O)', 'Oc1c(

In [220]:
# we also need all the additions vs the original and the reduced as emd matrixes
X_additions_original=[]
X_additions_reduced=[]
Y_additions=[]
red=reduced_incr_ter[0]
orig=original_incr_ter[0]
#add all but the testing set in
for i in range(5):
    A_pd=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_A.csv',header=0)
    B_pd=pd.read_csv('./Terpolymer_Splits/split_'+str(i)+'_B.csv',header=0)
    A_np=np.load('./Terpolymer_Splits/split'+str(i)+'_A_ys.npy')
    B_np=np.load('./Terpolymer_Splits/split'+str(i)+'_B_ys.npy')
    Y_additions+=[A_np,B_np]
    values_A=get_smiles_and_ratios_list(A_pd)
    values_B=get_smiles_and_ratios_list(B_pd)
    A_newpd=pd.DataFrame({'SMILES_List':values_A[0],'Ratio_List':values_A[1]})
    #print(A_newpd)
    B_newpd=pd.DataFrame({'SMILES_List':values_B[0],'Ratio_List':values_B[1]})
    emd_array_orig=get_EMD_array([A_newpd,B_newpd],orig)
    emd_array_red=get_EMD_array([A_newpd,B_newpd],red)
    X_additions_original+=[emd_array_orig[0],emd_array_orig[1]]
    X_additions_reduced+=[emd_array_red[0],emd_array_red[1]]
with open('./Terpolymer_Splits/X_list_of_EMD_for_additions_vs_orig.pkl',"wb") as f:
    dump(X_additions_original,f,protocol=5)
with open('./Terpolymer_Splits/X_list_of_EMD_for_additions_vs_red.pkl',"wb") as f:
    dump(X_additions_reduced,f,protocol=5)
with open('./Terpolymer_Splits/Y_list_of_np_for_additions.pkl',"wb") as f:
    dump(Y_additions,f,protocol=5)

In [243]:
#for each model set up, go through and train original models and incremental additions
#for each RF, there will have to be a new model trained with the total number of additional trees added for datapoint to keep total number of trees consistent
original_rf=[[8,4,8],[16,16,128],[16,128,512],[32,256,8]]
#reduced_rf selected for cross_val>0.81
reduced_rf=[[8,2,8],[16,4,128],[64,256,8],[256,4,256]]
with open('./240527_with_Terpolymers_Optimization Models/RF_original_training_incr_vs_retrain.csv','w',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Original Model Parameters','Retrain or Addition','Total Trees','Dataset Increase Number','Binary Polymers Score','Terpolymer Test Score','Combined Test Score'])
for i in range(len(original_rf)):
    #first open a file to record all the data
    C=original_rf[i]
    #load the original model
    with open("./240523_Optimization_Models/RF_os_estim_"+str(C[0])+"_depth_"+str(C[1])+"_leafnodes_"+str(C[2])+".pkl", "rb") as f:
        model=pickle.load(f)
    #score the original model with the testing parameters of interest
    Xs=original_emd_test[0]
    Ys=original_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])
    updated=model
    with open('./240527_with_Terpolymers_Optimization Models/RF_original_training_incr_vs_retrain.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow([str(C),'Basis',C[0],0,orig_score,ter_score,combined_score])
    for i in range(1,len(original_emd_test)):
        retrained_original_params=train_rf(C,[original_emd_train[i]],[original_incr_ter_ys[i-1]])
        C_new=[C[0]+(2*i),C[1],C[2]]
        retrained_incr_params=train_rf(C_new,[original_emd_train[i]],[original_incr_ter_ys[i-1]])
        updated=update_rf(updated,2*i,[X_additions_original[i-1]],[Y_additions[i-1]])
        models=[retrained_original_params,retrained_incr_params,updated]
        naming=['Retrain','Retrain','Addition']
        Cs=[C[0],C_new[0],C_new[0]]
        Xs=original_emd_test[i]
        Ys=original_test_ylist
        for j in range(3):
            if j==2:
                Xs=original_emd_test[0]
                Ys=original_test_ylist
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            else:
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            with open('./240527_with_Terpolymers_Optimization Models/RF_original_training_incr_vs_retrain.csv','a',newline='') as file:
                file_writer=csv.writer(file, delimiter=',')
                file_writer.writerow([str(C),naming[j],Cs[j],str(i),score_orig,score_ter,score_combo])




In [246]:
with open('./240527_with_Terpolymers_Optimization Models/RF_reduced_training_incr_vs_retrain.csv','w',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Original Model Parameters','Retrain or Addition','Total Trees','Dataset Increase Number','Binary Polymers Score','Terpolymer Test Score','Combined Test Score'])
for i in range(len(reduced_rf)):
    #first open a file to record all the data
    C=reduced_rf[i]
    #load the original model
    with open("./240523_Optimization_Models/RF_rl_estim_"+str(C[0])+"_depth_"+str(C[1])+"_leafnodes_"+str(C[2])+".pkl", "rb") as f:
        model=pickle.load(f)
    #score the original model with the testing parameters of interest
    Xs=reduced_emd_test[0]
    Ys=reduced_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])
    updated=model
    with open('./240527_with_Terpolymers_Optimization Models/RF_reduced_training_incr_vs_retrain.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow([str(C),'Basis',C[0],0,orig_score,ter_score,combined_score])
    for i in range(1,len(reduced_emd_test)):
        retrained_original_params=train_rf(C,[reduced_emd_train[i]],[reduced_incr_ter_ys[i-1]])
        C_new=[C[0]+(2*i),C[1],C[2]]
        retrained_incr_params=train_rf(C_new,[reduced_emd_train[i]],[reduced_incr_ter_ys[i-1]])
        updated=update_rf(updated,2*i,[X_additions_reduced[i-1]],[Y_additions[i-1]])
        models=[retrained_original_params,retrained_incr_params,updated]
        naming=['Retrain','Retrain','Addition']
        Cs=[C[0],C_new[0],C_new[0]]
        Xs=reduced_emd_test[i]
        Ys=reduced_test_ylist
        for j in range(3):
            if j==2:
                Xs=reduced_emd_test[0]
                Ys=reduced_test_ylist
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            else:
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            with open('./240527_with_Terpolymers_Optimization Models/RF_reduced_training_incr_vs_retrain.csv','a',newline='') as file:
                file_writer=csv.writer(file, delimiter=',')
                file_writer.writerow([str(C),naming[j],Cs[j],str(i),score_orig,score_ter,score_combo])

In [253]:
#now we do it again for the linear functions
#here we don't have the problem of of model changing size when we update for more data

original_SDG=[['hinge',0.1],['log_loss',0.01],['perceptron',0.001]]
#reduced SDG selected for
reduced_SDG=[['hinge',0.1],['log_loss',0.1],['perceptron',0.0001]]

with open('./240527_with_Terpolymers_Optimization Models/SDG_original_training_incr_vs_retrain.csv','w',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Original Model Parameters','Retrain or Addition','Dataset Increase Number','Binary Polymers Score','Terpolymer Test Score','Combined Test Score'])
for i in range(len(original_SDG)):
    #first open a file to record all the data
    C=original_SDG[i]
    #load the original model
    with open("./240523_Optimization_Models/SGD_os"+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "rb") as f:
        model=pickle.load(f)
    #score the original model with the testing parameters of interest
    Xs=original_emd_test[0]
    Ys=original_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])
    updated=model
    with open('./240527_with_Terpolymers_Optimization Models/SDG_original_training_incr_vs_retrain.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow([str(C),'Basis',0,orig_score,ter_score,combined_score])
    for i in range(1,len(original_emd_test)):
        retrained_original_params=train_lin(C,[original_emd_train[i]],[original_incr_ter_ys[i-1]])
        updated=update_lin(updated,X_additions_original[i-1],Y_additions[i-1])
        models=[retrained_original_params,updated]
        naming=['Retrain','Addition']
        Xs=original_emd_test[i]
        Ys=original_test_ylist
        for j in range(2):
            if j==1:
                Xs=original_emd_test[0]
                Ys=original_test_ylist
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            else:
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            with open('./240527_with_Terpolymers_Optimization Models/SDG_original_training_incr_vs_retrain.csv','a',newline='') as file:
                file_writer=csv.writer(file, delimiter=',')
                file_writer.writerow([str(C),naming[j],str(i),score_orig,score_ter,score_combo])



In [254]:
reduced_SDG=[['hinge',0.1],['log_loss',0.1],['perceptron',0.0001]]

with open('./240527_with_Terpolymers_Optimization Models/SDG_reduced_training_incr_vs_retrain.csv','w',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow(['Original Model Parameters','Retrain or Addition','Dataset Increase Number','Binary Polymers Score','Terpolymer Test Score','Combined Test Score'])
for i in range(len(reduced_SDG)):
    #first open a file to record all the data
    C=reduced_SDG[i]
    #load the original model
    with open("./240523_Optimization_Models/SGD_rl"+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "rb") as f:
        model=pickle.load(f)
    #score the original model with the testing parameters of interest
    Xs=reduced_emd_test[0]
    Ys=reduced_test_ylist
    orig_score=model.score(Xs[0],Ys[0])
    ter_score=model.score(Xs[1],Ys[1])
    combined_score=model.score(Xs[2],Ys[2])
    updated=model
    with open('./240527_with_Terpolymers_Optimization Models/SDG_reduced_training_incr_vs_retrain.csv','a',newline='') as file:
        file_writer=csv.writer(file, delimiter=',')
        file_writer.writerow([str(C),'Basis',0,orig_score,ter_score,combined_score])
    for i in range(1,len(reduced_emd_test)):
        retrained_original_params=train_lin(C,[reduced_emd_train[i]],[reduced_incr_ter_ys[i-1]])
        updated=update_lin(updated,X_additions_reduced[i-1],Y_additions[i-1])
        models=[retrained_original_params,updated]
        naming=['Retrain','Addition']
        Xs=reduced_emd_test[i]
        Ys=reduced_test_ylist
        for j in range(2):
            if j==1:
                Xs=reduced_emd_test[0]
                Ys=reduced_test_ylist
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            else:
                score_orig=models[j].score(Xs[0],Ys[0])
                score_ter=models[j].score(Xs[1],Ys[1])
                score_combo=models[j].score(Xs[2],Ys[2])
            with open('./240527_with_Terpolymers_Optimization Models/SDG_reduced_training_incr_vs_retrain.csv','a',newline='') as file:
                file_writer=csv.writer(file, delimiter=',')
                file_writer.writerow([str(C),naming[j],str(i),score_orig,score_ter,score_combo])



In [255]:
additional_datapoints=[0]
total_datapoints_added=[0]
start=0
for i in range(len(Y_additions)):
    adding=Y_additions[i].size
    start=start+adding
    additional_datapoints+=[adding]
    total_datapoints_added+=[start]
print(additional_datapoints)
print(total_datapoints_added)

[0, 25, 23, 25, 24, 25, 25, 28, 27, 30, 31]
[0, 25, 48, 73, 97, 122, 147, 175, 202, 232, 263]


In [257]:
print(len(os_train_pd.index))
print(len(rl_train_pd.index))


546
197


In [258]:
print(546*np.ones(11))

[546. 546. 546. 546. 546. 546. 546. 546. 546. 546. 546.]


In [259]:
print(np.array(total_datapoints_added)/(np.array(total_datapoints_added)+546*np.ones(11)))


[0.         0.04378284 0.08080808 0.11793215 0.15085537 0.18263473
 0.21212121 0.24271845 0.27005348 0.29820051 0.32509271]


In [ ]:
C=["log_loss",0.01]
with open("./240523_Optimization_Models/SGD_rl"+"_loss_"+str(C[0])+"_alpha_"+str(C[1])+".pkl", "rb") as f:
        model=pickle.load(f)
updated=update_lin(updated,X_additions_original[0],Y_additions[0])

